In [15]:
import json
import pandas as pd
import os
from dotenv import load_dotenv
from langchain_google_genai import GoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [16]:
# Load environment variables
load_dotenv()
api_key = os.getenv("MY_KEY")

In [17]:
# Initialize the language model
llm = GoogleGenerativeAI(model="models/gemini-1.0-pro", google_api_key=api_key, temperature=1.0)

In [18]:
# Define the desired JSON format
RESPONSE_JSON = """
{
    "questions": {
        "mcq": [
            {
                "question": "What was the primary reason for the residents of Taj Nagar to build their own railway station?",
                "options": [
                    {"a": "They wanted to attract more businesses to their village."},
                    {"b": "The railway authorities refused to build a station due to lack of funds."},
                    {"c": "They wanted to prove their ability to self-govern."},
                    {"d": "They were dissatisfied with the existing railway stations nearby."}
                ],
                "correct_answer": "b"
            },
            {
                "question": "What was the main initiative undertaken by Mera Gao Power (MGP) to address the power shortage in Sitapur?",
                "options": [
                    {"a": "Installing large-scale solar power plants."},
                    {"b": "Providing subsidies for purchasing solar panels."},
                    {"c": "Building solar power mini-grids at the village level."},
                    {"d": "Advocating for government-funded electrification projects."}
                ],
                "correct_answer": "c"
            },
            {
                "question": "What is the main objective of the women's unit formed at the Palakkad District Public Library?",
                "options": [
                    {"a": "To promote literacy and empowerment among women."},
                    {"b": "To organize cultural events and workshops for women."},
                    {"c": "To provide a safe and comfortable space for women to socialize."},
                    {"d": "To advocate for increased government funding for libraries."}
                ],
                "correct_answer": "a"
            },
            {
                "question": "What unique feature distinguishes the railway station built by the residents of Taj Nagar?",
                "options": [
                    {"a": "It is the only station in India built entirely with private funding."},
                    {"b": "It is the first station built using sustainable materials."},
                    {"c": "It is the smallest railway station in India."},
                    {"d": "It is the only station built by a village community."}
                ],
                "correct_answer": "a"
            },
            {
                "question": "What is a significant benefit of using solar power as a source of light, as highlighted in the case of Sitapur?",
                "options": [
                    {"a": "It reduces the dependence on fossil fuels."},
                    {"b": "It creates employment opportunities in the renewable energy sector."},
                    {"c": "It improves air quality and reduces pollution."},
                    {"d": "It is more affordable than other forms of electricity."}
                ],
                "correct_answer": "c"
            }
        ],
        "short_note": [
            {"question": "Describe the steps taken by the residents of Taj Nagar to build their own railway station."},
            {"question": "Explain the approach adopted by Mera Gao Power (MGP) to provide electricity to rural households in Sitapur."},
            {"question": "Discuss the role of the Palakkad District Public Library in empowering women."},
            {"question": "What were the challenges faced by the residents of Taj Nagar in their quest for a railway station?"},
            {"question": "What are the benefits of using solar power for rural communities, as illustrated in the case of Sitapur?"}
        ],
        "blank": [
            {"question": "The residents of Taj Nagar pooled in ______ rupees to build their own railway station.", "answer": "twenty-one lakh"},
            {"question": "Mera Gao Power provides ______ LED lights and a mobile charging point to each paying household.", "answer": "two"},
            {"question": "The Palakkad District Public Library opened its halls for workshops in ______ or gardening.", "answer": "home economics"},
            {"question": "The women's unit at the Palakkad District Public Library discussed the importance of separate ______ for women.", "answer": "reading rooms"},
            {"question": "The first railway station in India on which the railway did not have to spend a single rupee, started operations on ______.", "answer": "7 January 2010"}
        ]
    }
}
"""

In [19]:
# Define the prompt template
PROMPT_TEMPLATE = """
Text: {text}
You are an expert MCQ maker. Given the above text, create {count} multiple choice questions, {count} short note questions, and {count} fill-in-the-blank questions.You should generate {difficulty} Questions. Questions shall be unique each time you generate.
Ensure to format your response as follows:
{response_json}
"""


In [20]:
# Create the prompt template
prompt_template = PromptTemplate(
    input_variables=["text"],
    template=PROMPT_TEMPLATE
)

In [21]:
# Create the LLMChain
chain = LLMChain(
    llm=llm,
    prompt=prompt_template
)

In [22]:
# Define the text for quiz generation
TEXT = "Tracks to Taj Nagar For twenty five years, residents of Taj Nagar village near Gurgaon lobbied for a railway station in their village. When their demand was not met, the villagers decided to take matters into their own hands. They pooled in twenty one lakh rupees and built a railway station on their own. Most of the three thousand people living in the village are farmers. But such was the burning desire to have a station in the village, everybody contributed according to their capacity. Ranging from three thousand rupees to seventy five thousand rupees. \"They donated money for the station and we started the construction in January 2008.\" said Ranjit Singh, a former village sarpanch. \"There are a large number of people in the village who need to go to Gurgaon, Delhi and Rewar. There are students who go to colleges. Till now, we had to either go to Halimandi or Patli to catch a train. Both the stations are six kilometers away from Taj Nagar. We thought when the railway lines passed through the village we would have a station here. But that didn\'t happen. So we raised the demand in 1982 and have been continuously asking for it, but the railways told us that they did not have funds. So, finally we decided to craft our own destiny,\" said Hukamchand, a member of the committee. As a result, the panchayat passed a resolution in 2008, saying that since the railway was not able to build a station for them, they would do it for themselves and with their own money! Soon, an eleven member team was formed and the team started collecting money from villagers. How would you solve these problems with minimum help from others? AGAINST THE ODDS On 7 January 2010, as a result of their efforts, the first railway station in the country on which the railway did not have to spend a single rupee, started operations. Sitapur\'s Light In rural Uttar Pradesh, over sixty percent of households are without power. Sitapur district is one such place with no power. A small social enterprise called Mera Gao Power (MGP) is trying to change things. They are putting two solar panels at a time. In just over a year, MGP has connected more than 3,500 customers to solar power mini- grids at a village level. Village by village, MGP is building a network of low cost solar micro-grids that provide two LED lights and a mobile charging point to all paying house holds at a cost of twenty five rupees per week. That is cheaper than kerosene which can cost almost double across a month. Solar power, as a \'smokeless\' source of light, comes with added benefits to customer health. Installing a micro-grid is a grand event in the village and every one gets involved. In the village of Damdampurawa, the team maps the village house by house beneath the scorching mid-day sun, working out where to place each wire so as to connect customer to the power source. Some house holders join in while others look on, calling out orders or watching the curious proceedings wide- eyed. The roof of a sturdy, brick-walled home in each village is always chosen as the site for the panels and the battery. Azaz, one of the company\'s first electrician to be recruited from the local district block of Reusa, installs English (S.L.), Std. 10 the panel in a southerly direction to capture as much sun light as possible. \"We\'re saving our environment with these lights, and there\'s no pollution in our homes either,\" says a farmer from the village. \"New businesses are starting to emerge amongst the customers too,\" says another. \"In one village, customers are using the light to weave saris by night. In another, one man now has a night business making plastic tablecloth,\" he says. \"Tt\'s nice to have light while we cook and eat. Our children are also studying more now!\" Palakkad\'s Public Library In kerala The Palakkad District Public Library has been up and running since September 2013. It is a fine modern library, a center for information, knowledge, wisdom, cultural activities, research and reference. But it has recently been in the news for different reasons. A third of its thousand members are women. These women, supported by the shared space the library offered them, launched a women\'s unit in February, 2014. The unit got together to discuss methods of empowering women. Glossary The library opened its halls for film screening, workshops in home economics or gardening, child care or the arts, and for women to get help in managing family conflicts, legal disputes and professional problems. The secretary of the library pointed out that through reading, women would realize their own strength and forge a unity. It was noted that the lending libraries of earlier times were disappearing and the present rural reading rooms were too often full of only male readers. The unit discussed that if the once well-read women of Kerala continued to squander their hours in front of television, it would encourage a climate in which women are afraid to go out after dark. So, the unit has formulated plans on opening separate reading rooms for women. Palakkad\'s district library stands tall asa beacon to encourage women\'s empowerment through classes, clubs, workshops and reading rooms. And then, there are the books, which will provide the women the strength they need to make good use of these opportunities."

In [23]:
# Generate the quiz
response = chain.invoke({"text": TEXT,"response_json":RESPONSE_JSON, "count": 5,"difficulty":"Hard" })
print(response)

{'text': '{\n    "questions": {\n        "mcq": [\n            {\n                "question": "What motivated the villagers of Taj Nagar to build their own railway station?",\n                "options": [\n                    {"a": "Lack of funds from the railway authorities."},\n                    {"b": "Improved access to transportation and education."},\n                    {"c": "Increased tourism and economic development."},\n                    {"d": "All of the above."}\n                ],\n                "correct_answer": "d"\n            },\n            {\n                "question": "What is the significance of Mera Gao Power\'s (MGP) approach to providing electricity in Sitapur?",\n                "options": [\n                    {"a": "It reduces the environmental impact through renewable energy."},\n                    {"b": "It provides affordable and reliable power to rural households."},\n                    {"c": "It promotes community participation in electrificati

In [24]:
# Extract and clean the quiz string
quiz_str = response.get("text")
if quiz_str is None:
    print("No quiz string received.")
else:
    print("Raw quiz string:", quiz_str)
    quiz_str = quiz_str.strip("```json").strip("```") if quiz_str else "{}"
    print("Cleaned quiz string:", quiz_str)

Raw quiz string: {
    "questions": {
        "mcq": [
            {
                "question": "What motivated the villagers of Taj Nagar to build their own railway station?",
                "options": [
                    {"a": "Lack of funds from the railway authorities."},
                    {"b": "Improved access to transportation and education."},
                    {"c": "Increased tourism and economic development."},
                    {"d": "All of the above."}
                ],
                "correct_answer": "d"
            },
            {
                "question": "What is the significance of Mera Gao Power's (MGP) approach to providing electricity in Sitapur?",
                "options": [
                    {"a": "It reduces the environmental impact through renewable energy."},
                    {"b": "It provides affordable and reliable power to rural households."},
                    {"c": "It promotes community participation in electrification projects.

In [25]:
# Validate and fix the JSON
def validate_json(json_str):
    try:
        return json.loads(json_str), None
    except json.JSONDecodeError as e:
        return None, f"JSON decode error: {e}"
    except Exception as e:
        return None, f"An unexpected error occurred: {e}"

In [26]:
# Validate the JSON
quiz_dict, error = validate_json(quiz_str)

if error:
    print(f"Error parsing JSON: {error}")
    quiz_dict = {'questions': {'mcq': [], 'short_note': [], 'blank': []}}
else:
    print("JSON is valid.")
    print("Parsed JSON:", quiz_dict)

JSON is valid.
Parsed JSON: {'questions': {'mcq': [{'question': 'What motivated the villagers of Taj Nagar to build their own railway station?', 'options': [{'a': 'Lack of funds from the railway authorities.'}, {'b': 'Improved access to transportation and education.'}, {'c': 'Increased tourism and economic development.'}, {'d': 'All of the above.'}], 'correct_answer': 'd'}, {'question': "What is the significance of Mera Gao Power's (MGP) approach to providing electricity in Sitapur?", 'options': [{'a': 'It reduces the environmental impact through renewable energy.'}, {'b': 'It provides affordable and reliable power to rural households.'}, {'c': 'It promotes community participation in electrification projects.'}, {'d': 'All of the above.'}], 'correct_answer': 'd'}, {'question': "What is a key focus of the women's unit established at the Palakkad District Public Library?", 'options': [{'a': 'Facilitating social interactions and community building.'}, {'b': "Providing resources for women'

In [27]:
# Prepare data for DataFrame
quiz_table_data = []
for question_type, questions in quiz_dict.get("questions", {}).items():
    if question_type == "mcq":
        for question in questions:
            mcq = question.get("question", "")
            options = " | ".join([f"{list(option.keys())[0]}: {list(option.values())[0]}" for option in question.get("options", [])])
            correct = question.get("correct_answer", "")
            quiz_table_data.append({
                "Question Type": "Multiple Choice",
                "Question": mcq,
                "Choices": options,
                "Correct Answer": correct
            })
    elif question_type == "short_note":
        for question in questions:
            short_note = question.get("question", "")
            quiz_table_data.append({
                "Question Type": "Short Note",
                "Question": short_note,
                "Choices": None,
                "Correct Answer": None
            })
    elif question_type == "blank":
        for question in questions:
            blank = question.get("question", "")
            correct = question.get("answer", "")
            quiz_table_data.append({
                "Question Type": "Fill in the Blank",
                "Question": blank,
                "Choices": None,
                "Correct Answer": correct
            })

In [28]:
# Convert to DataFrame
quiz_table_data
df = pd.DataFrame(quiz_table_data)
df.to_csv("Data_Science_Quiz.csv",index=False)
print(df)

        Question Type                                           Question  \
0     Multiple Choice  What motivated the villagers of Taj Nagar to b...   
1     Multiple Choice  What is the significance of Mera Gao Power's (...   
2     Multiple Choice  What is a key focus of the women's unit establ...   
3     Multiple Choice  What is a unique aspect of the railway station...   
4     Multiple Choice  What is a primary advantage of using solar pow...   
5          Short Note  Discuss the main challenges faced by the resid...   
6          Short Note  Explain the innovative solutions and community...   
7          Short Note  Analyze the role of the Palakkad District Publ...   
8          Short Note  Examine the broader implications of the self-s...   
9          Short Note  Evaluate the environmental and socio-economic ...   
10  Fill in the Blank  The construction of the Taj Nagar railway stat...   
11  Fill in the Blank  Mera Gao Power's (MGP) solar power mini-grids ...   
12  Fill in 